In [1]:
import pandas as pd
import numpy as np
# import math
from utils_env import *

In [2]:
Sb = 100.0                # MW/MVar/MVA
Vb = 12.66             # kV
Zb = Vb**2/Sb           # O
# Ib = Sb/(math.sqrt(3)*Vb)   # kA
V0 = 1.0 # p.u.
V_max = 1.05
V_min = 0.95
Big_M_FF = 40             # 单商品流的BigM
Big_M_V = 3               # 压降松弛的BigM
BigM_SC = 2

NT = 5

In [3]:
Bus_Data=pd.read_excel('Case_33BW_Data.xlsx',sheet_name='Bus').to_numpy()
Branch_Data=pd.read_excel('Case_33BW_Data.xlsx',sheet_name='Branch').to_numpy()
DG_Data = pd.read_excel('Case_33BW_Data.xlsx',sheet_name='DG').to_numpy()

In [4]:
N_Bus = 33
Pd_Data = Bus_Data[:, 1] / Sb
Pd_Data_all = np.sum(Pd_Data)
Qd_Data = Bus_Data[:, 2] / Sb
Qd_Data_all = np.sum(Qd_Data)
Pd_ratio = Pd_Data / Pd_Data_all  # Active load ratio
Qd_ratio = Qd_Data / Qd_Data_all  # Reactive load ratio

N_Branch = Branch_Data.shape[0]
N_TL = np.sum(Branch_Data[:, 6] == 1).item()  # 必须加上.item()，否则返回的是array([x]) 导致后面传入julia进程数据类型错误
N_NL = np.sum(Branch_Data[:, 6] == 0).item()

Branch_start = Branch_Data[:, 1].astype(int)  #每条线路的起点和终点
Branch_end = Branch_Data[:, 2].astype(int)

# LS_Mask = make_mask(N_Bus,N_Branch,Branch_start ) 
# LE_Mask = make_mask(N_Bus,N_Branch,Branch_end )

pIn = make_inc_matrix(Branch_start, Branch_end)
pInn = np.copy(pIn)
pInn[pInn > 0] = 0

R_Branch0 = Branch_Data[:, 3] / Zb
X_Branch0 = Branch_Data[:, 4] / Zb
SZ_Branch0 = R_Branch0 ** 2 + X_Branch0 ** 2
S_Branch0 = Branch_Data[:, 5] / Sb

N_DG = DG_Data.shape[0]
DataDN_IndDG = DG_Data[:, 1]
DataDN_IndBSDG = DG_Data[DG_Data[:, 6] == 1, 1]
DataDN_IndNMDG = DG_Data[DG_Data[:, 6] == 0, 1]
N_BSDG = DataDN_IndBSDG.shape[0]
# N_NMDG = DataDN_IndNMDG.shape[0]
P_DG_max0 = DG_Data[:, 2] / Sb
P_DG_min0 = DG_Data[:, 3] / Sb
Q_DG_max0 = DG_Data[:, 4] / Sb
Q_DG_min0 = DG_Data[:, 5] / Sb
DG_Mask = make_mask(N_Bus, N_DG, DataDN_IndDG)
BSDG_Mask = make_mask(N_Bus, N_BSDG, DataDN_IndBSDG)
# NMDG_Mask = make_mask(N_Bus, N_NMDG, DataDN_IndNMDG)

load_pec = 1.0
Pd_all = Pd_Data_all * load_pec
Qd_all = Qd_Data_all * load_pec
X_tieline0 = np.zeros(N_TL)
dmg_list = [6, 11, 29, 32]

Pd = Pd_all * np.tile(Pd_ratio, (NT, 1)).T
Qd = Qd_all * np.tile(Qd_ratio, (NT, 1)).T

R_Branch = np.tile(R_Branch0, (NT, 1)).T
X_Branch = np.tile(X_Branch0, (NT, 1)).T
S_Branch = np.tile(S_Branch0, (NT, 1)).T

P_DG_max = np.tile(P_DG_max0, (NT, 1)).T
P_DG_min = np.tile(P_DG_min0, (NT, 1)).T
Q_DG_max = np.tile(Q_DG_max0, (NT, 1)).T
Q_DG_min = np.tile(Q_DG_min0, (NT, 1)).T



a = np.ones((N_NL, NT))

for dmg in dmg_list:
    a[dmg-1, :] = 0

X_rec0 = np.zeros(33)
X_tieline_input = np.array([1, 1, 1, 0, 0])
Q_svc_input = np.array([0.002, 0.002, 0.000, 0.002, 0.002, 0.000])

args_expert = (NT, N_Branch, N_TL, N_NL, N_Bus, pIn, N_DG, DG_Mask, R_Branch, X_Branch, Big_M_V, V0,
        V_min, V_max, Pd, Qd, S_Branch, P_DG_min, P_DG_max, Q_DG_min, Q_DG_max, BigM_SC, BSDG_Mask,
        Big_M_FF, a, X_tieline0)

args_step = (1, N_Branch, N_TL, N_NL, N_Bus, pIn, N_DG, DG_Mask, R_Branch[:, 0], X_Branch[:, 0], Big_M_V, V0,
        V_min, V_max, Pd[:, 0], Qd[:, 0], S_Branch[:, 0], P_DG_min[:, 0], P_DG_max[:, 0], Q_DG_min[:, 0],
        Q_DG_max[:, 0], BSDG_Mask, Big_M_FF, a[:, 0], X_rec0, X_tieline_input, Q_svc_input)

In [5]:
from julia import Main as jl
jl.include("system_test.jl")
jl.make_opf(args_expert,args_step)
# 第一次初始化 5700G大约30s

-1.4109000000000003

In [6]:
jl.opf()
# 初始化完成后就很快了

-1.4109000000000003

In [7]:
# 以下为测试修改a (env.rest)后，专家模型的变化
a_input = np.ones((N_NL, NT))
dmg_list_new = [11,21] # 先随便定义一个
for dmg in dmg_list_new:
    a_input[dmg-1, :] = 0

In [8]:
jl.mod_a(a_input)

-1.6652500000000001